In [11]:
from pdf2image import convert_from_path

In [12]:
pages = convert_from_path(r'docs\patient_details\pd_1.pdf', poppler_path=r'C:\poppler-22.04.0\Library\bin')

In [13]:
pages

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1867x2000 at 0x239D2A01E20>,
 <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1398 at 0x239D2A01AC0>]

In [14]:
pages[0].show()

In [15]:
import pytesseract

In [16]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(pages[0], lang='eng')
print(text)

47/12/2020

Patient Medical Record

Patient Information Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight
9264 Ash Dr 95
New York City, 10005 .
United States Height:
190
In Case of Emergency
m _ eee ee
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History

Chicken Pox (Varicella):

IMMUNE IMMUNE
Have you had the Hepatitis B vaccination?

No
List any Medical Problems (asthma, seizures, headaches):

Migraine


In [17]:
import numpy as np
import cv2
from PIL import Image

def preprocess_image(img):
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    processed_image = cv2.adaptiveThreshold(
        resized,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        61,
        11
    )
    return processed_image

In [18]:
img = preprocess_image(pages[0])
Image.fromarray(img).show()

In [19]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
text = pytesseract.image_to_string(img, lang='eng')
print(text)

17/12/2020

Patient Medical Record . : :

Patient Information

Birth Date
Kathy Crawford May 6 1972
(737) 988-0851 Weight:
9264 Ash Dr 95
New York City, 10005 a
United States Height:
190
In Case of Emergency
ee oe
Simeone Crawford 9266 Ash Dr
New York City, New York, 10005
Home phone United States
(990) 375-4621
Work phone
Genera! Medical History
I i
Chicken Pox (Varicella): Measies:
IMMUNE IMMUNE

Have you had the Hepatitis B vaccination?

No

List any Medical Problems (asthma, seizures, headaches):

Migraine


In [20]:
import re

In [21]:
pattern = 'Patient Information(.*?)\(\d{3}\)'
matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['\n\nBirth Date\nKathy Crawford May 6 1972\n']

In [22]:
matches[0].strip()

'Birth Date\nKathy Crawford May 6 1972'

In [23]:
match = matches[0].replace("Birth Date","").strip()
match

'Kathy Crawford May 6 1972'

In [24]:
pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
date_matches = re.findall(pattern, match)
date = date_matches[0][0]
date

'May 6 1972'

In [25]:
match.replace(date, '').strip()

'Kathy Crawford'

In [26]:
def remove_noise_from_name(name):
    name = name.replace('Birth Date','').strip()
    date_pattern = '((Jan|Feb|March|April|May|June|July|Aug|Sep|Oct|Nov|Dec)[ \d]+)'
    date_matches = re.findall(date_pattern, name)
    
    if date_matches:
        date = date_matches[0][0]
        name = name.replace(date, '').strip()
    
    return name

In [27]:
name = '\n\nBirth Date\nKathy Crawford May 6 1972\n'
name = remove_noise_from_name(name)
name

'Kathy Crawford'

In [28]:
pattern = 'Patient Information(.*?)(\(\d{3}\) \d{3}-\d{4})'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

[('\n\nBirth Date\nKathy Crawford May 6 1972\n', '(737) 988-0851')]

In [29]:
matches[0][1]

'(737) 988-0851'

In [30]:
pattern = 'Have you had the Hepatitis B vaccination\?.*(Yes|No)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches

['No']

In [32]:
pattern = 'List any Medical Problems \(asthma, seizures, headaches\):(.*)'

matches = re.findall(pattern, text, flags=re.DOTALL)
matches[0].strip()

'Migraine'